In [1]:
#Autor: Alonso Melgarejo
#Contacto: alonsoraulmgs@gmail.com
#Copyright: Big Data Academy

# Librerías

In [2]:
#Importamos las librerías estándar
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import glob
import os

In [3]:
#Importamos las librerías de Keras
from keras.utils.np_utils import to_categorical
from keras.preprocessing import image
from keras.models import Sequential
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import MaxPool2D, AvgPool2D
from keras.layers import Activation
from keras.layers import Dense

# Conexión al repositorio de datos

In [4]:
#Nos conectamos a Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Funciones utilitarias

In [5]:
def leer_carpeta_de_imagenes_como_tensor(ruta):
  tensores = []

  #Obtenemos las rutas de las imágenes
  rutas_imagenes = sorted(glob.glob(ruta+'/*'))

  index = 0
  for ruta in rutas_imagenes:
    #Leemos la imagen
    imagen = image.load_img(
      ruta, 
      grayscale=False,
      color_mode="grayscale",
      target_size = (28, 28),
      interpolation="nearest"
    )

    #La tensorizamos
    tensor = np.array([image.img_to_array(imagen)])

    #La agregamos a la lista de tensores
    if index == 0:
      tensores = tensor
    else:
      tensores = np.concatenate((tensores, tensor))

    index = index + 1

  #La retornamos
  return tensores

# Lectura de tensores

In [6]:
from keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data('/tmp/mnist.npz')

11501568/11490434 [==============================] - 0s 0us/step


# Procesamiento de features

In [7]:
#PROCESAMIENTO DE LOS FEATURES DE ENTRENAMIENTO 

In [8]:
#Si no está en forma tensorial [imagenes, pixelX, pixelY, canales], deberemos adaptarlo
x_train.shape

(60000, 28, 28)

In [9]:
#Le agregamos la dimensión faltante
x_tensor_train = x_train.reshape(-1, 28, 28, 1)
x_tensor_train.shape

(60000, 28, 28, 1)

In [10]:
#Escalamos los datos
x_tensor_train_scale = x_tensor_train.astype('float32') / 255.0
x_tensor_train_scale.shape

(60000, 28, 28, 1)

In [11]:
#PROCESAMIENTO DE LOS FEATURES DE VALIDACIÓN 

In [12]:
#Si no está en forma tensorial [imagenes, pixelX, pixelY, canales], deberemos adaptarlo
x_test.shape

(10000, 28, 28)

In [13]:
#Le agregamos la dimensión faltante
x_tensor_test = x_test.reshape(-1, 28, 28, 1)
x_tensor_test.shape

(10000, 28, 28, 1)

In [14]:
#Escalamos los datos
x_tensor_test_scale = x_tensor_test.astype('float32') / 255.0
x_tensor_test_scale.shape

(10000, 28, 28, 1)

# Procesamiento de labels

In [15]:
#PROCESAMIENTO DE LOS LABELS DE ENTRENAMIENTO 

In [16]:
#Los labels tienen que estar categorizados
y_train

array([5, 0, 4, ..., 5, 6, 8], dtype=uint8)

In [17]:
#Categorizamos los labels
y_train_categorico = to_categorical(y_train)
y_train_categorico.shape

(60000, 10)

In [18]:
#PROCESAMIENTO DE LOS LABELS DE VALIDACION 

In [19]:
#Los labels tienen que estar categorizados
y_test

array([7, 2, 1, ..., 4, 5, 6], dtype=uint8)

In [20]:
#Categorizamos los labels
y_test_categorico = to_categorical(y_test)
y_test_categorico.shape

(10000, 10)

# Construcción de Red Convolucional

In [21]:
#Instaciamos un modelo vacío
model = Sequential()

In [22]:
#Agregamos una capa, de 16 filtros, cada filtro de 4x4
#El input es una imagen de 28x28 pixeles de 1 canal de color
model.add(Conv2D(16, (4, 4), input_shape=(28, 28, 1)))

In [23]:
model.add(Conv2D(4, (4, 4)))

# Vectorización del tensor

In [24]:
#Vectorizamos los patrones
model.add(Flatten())

# Red neuronal para clasificación

In [25]:
#Agregamos capas de nuestra red neuronal, 128 neuronas
model.add(Dense(128, activation='relu'))

In [26]:
#Como estamos clasificando, agregamos una capa de salida de 10 neuronas con softmax
model.add(Dense(10, activation='softmax'))

# Compilación del modelo de Deep Learning (Red Convolucional + Red Neuronal)

In [27]:
#Compilamos
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

# Entrenamiento del modelo

In [28]:
#AL ejecutarlo, notamos que se entrena muy lento
#Esto se debe a la gran cantidad de información que se está procesando, matricialmente cada imagen es de 28x28, 
model.fit(x_tensor_train_scale, y_train_categorico, batch_size=128, epochs=2, verbose=1, validation_split=0.3)

Epoch 1/2
329/329 [==============================] - 34s 101ms/step - loss: 0.2510 - accuracy: 0.9253 - val_loss: 0.2213 - val_accuracy: 0.9356
Epoch 2/2
329/329 [==============================] - 33s 101ms/step - loss: 0.1061 - accuracy: 0.9684 - val_loss: 0.1119 - val_accuracy: 0.9676


# Evaluación del modelo

In [29]:
#Evaluamos el modelo
#Notamos que está acertando en el 97% de los casos
model.evaluate(x_tensor_test_scale, y_test_categorico)

313/313 [==============================] - 2s 6ms/step - loss: 0.0977 - accuracy: 0.9716


[0.09774048626422882, 0.9715999960899353]

# Uso del modelo

In [30]:
#Leemos las imagenes como tensores
tensores = leer_carpeta_de_imagenes_como_tensor('/content/drive/MyDrive/Data/MNIST')
tensores.shape

(5, 28, 28, 1)

In [31]:
#Usamos el modelo para predecir
predicciones_labels = model.predict(tensores)
predicciones_labels

array([[0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]], dtype=float32)

In [32]:
#Obtenemos el índice del array de predicciones para saber a qué categoría pertenece
predicciones = predicciones_labels.argmax(axis = 1)
predicciones

array([3, 0, 4, 1, 9])